In [24]:
import numpy as np
from scipy.stats import norm

#input values setting
s0 = 50
k = 55
r = 0.1
q = 0.03
T = 0.5
n = 100
convergence_criterion = 1e-9
european_call = 2.5
american_put = 6.5
max_iters=10000

In [78]:
#Extra Bonus1
#Bisection Method
def bisection(fx, a, b, tol=1e-9, max_iters=10000):
  fa = fx(a)
  fb = fx(b)
  if fa*fb > 0:
    print("f does not have opposite sign at endpoints")
  
  c = (a+b)/2
  fc = fx(c)
  iter = 0
  while (iter<max_iters) and (abs(fc)>tol):
    iter = iter+1
    if fa*fc < 0:
      b = c
      fb = fc
    else:
      a = c
      fa = fc

    c = (a+b)/2
    fc = fx(c)
  return 100*c
  
#Black Scholes Formula
def BS(s0, k, r, q, sigma, T):
    d1 = (np.log(s0/k)+(r-q+0.5*sigma*sigma)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(s0/k)+(r-q-0.5*sigma*sigma)*T)/(sigma*np.sqrt(T))
    call = s0*np.exp(-q*T)*(norm.cdf(d1))-k*np.exp(-r*T)*(norm.cdf(d2))
    put = k*np.exp(-r*T)*norm.cdf(-d2)-s0*np.exp(-q*T)*norm.cdf(-d1)
    return call, put

def implied_volatility_BS_European_call(sigma):
    result = BS(s0, k, r, q, sigma, T)[0] - european_call
    return result

#CRR binomial tree
def binomial_tree(s0, k, r, q, T, n, sigma, type):
    dt = T/n
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    p = (np.exp((r-q)*dt)-d)/(u-d)
    s = np.zeros((n+1, n+1))
    c = np.zeros((n+1, n+1))
    put = np.zeros((n+1, n+1))
    s[0, 0] = s0

    for i in range(0, n+1):
        for j in range(i, n+1):
            s[i, j] = s0*(u**(j-i))*(d**i)
            c[i, n] = max(0, s[i, n]-k)
            put[i, n] = max(0, k-s[i, n])

    for j in range(n-1, -1, -1):
        for i in range(0, j+1):
            c[i, j] = np.exp(-r*dt)*(p*c[i, j+1]+(1-p)*c[i+1, j+1])
            put[i, j] = np.exp(-r*dt)*(p*put[i, j+1]+(1-p)*put[i+1, j+1])
            if type == 'European':
                pass
            elif type == 'American':
                c[i, j] = max(c[i, j], (s[i, j]-k))
                put[i, j] = max(put[i, j], (k-s[i, j]))
    return c[0, 0], put[0, 0]

def implied_volatility_CRR_European_call(sigma):
    result = binomial_tree(s0, k, r, q, T, n, sigma, 'European')[0]- european_call
    return result

def implied_volatility_CRR_American_put(sigma):
    result = binomial_tree(s0, k, r, q, T, n, sigma, 'American')[1]- american_put
    return result

In [79]:
print('The implied volatility of European call(Black Scholes) is', bisection(implied_volatility_BS_European_call, 0.2, 0.4, convergence_criterion, max_iters), '%')
print('The implied volatility of European call(Binomial Tree) is', bisection(implied_volatility_CRR_European_call, 0.2, 0.4, convergence_criterion, max_iters), '%')
print('The implied volatility of American put(Binomial Tree) is', bisection(implied_volatility_CRR_American_put, 0.2, 0.4, convergence_criterion, max_iters), '%')

The implied volatility of European call(Black Scholes) is 26.839312370866548 %
The implied volatility of European call(Binomial Tree) is 26.78457652218641 %
The implied volatility of American put(Binomial Tree) is 30.49048574641347 %


In [81]:
#Newtons method
def newtons_method(x0, fx, dfx, tol=1e-9, max_iters=10000):
    x = x0
    iter = 0
    while (abs(x-(x-fx(x)/dfx(x)))>tol) and (iter<max_iters) and(abs(dfx(x))>tol):
        iter = iter+1
        x = x-fx(x)/dfx(x)
    return 100*x

def derivative_of_BS_European_call(sigma):
    h = 1e-9
    return (implied_volatility_BS_European_call(sigma+h)-implied_volatility_BS_European_call(sigma-h))/(2*h)

def derivative_of_CRR_European_call(sigma):
    h = 1e-9
    return (implied_volatility_CRR_European_call(sigma+h)-implied_volatility_CRR_European_call(sigma-h))/(2*h)

def derivative_of_CRR_American_put(sigma):
    h = 1e-9
    return (implied_volatility_CRR_American_put(sigma+h)-implied_volatility_CRR_American_put(sigma-h))/(2*h)



In [82]:
print('The implied volatility of European call(Black Scholes) is', newtons_method(0.2, implied_volatility_BS_European_call, derivative_of_BS_European_call, tol=1e-9, max_iters=10000), '%')
print('The implied volatility of European call(Binomial Tree) is', newtons_method(0.2, implied_volatility_CRR_European_call, derivative_of_CRR_European_call, tol=1e-9, max_iters=10000), '%')
print('The implied volatility of American put(Binomial Tree) is', newtons_method(0.2, implied_volatility_CRR_American_put, derivative_of_CRR_American_put, tol=1e-9, max_iters=10000), '%')

The implied volatility of European call(Black Scholes) is 26.839312372363693 %
The implied volatility of European call(Binomial Tree) is 26.78457652031302 %
The implied volatility of American put(Binomial Tree) is 30.490485749680207 %
